# Importações

In [ ]:
import os
import pandas as pd
from IPython.display import Markdown, display
import mysql.connector as ms
from dotenv import load_dotenv
load_dotenv()

# Ambiente de Execução: Colab X VSCode

In [ ]:
# # Usar no Google Colab
# from google.colab import auth
# auth.authenticate_user()
# path_folder_gold_dim = '/content/'
# path_folder_gold_fact = '/content/'

In [ ]:
# Usar no VSCode
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../gcp_key.json"
path_folder_gold_dim = f'../data/temp/gold/dim/'
path_folder_gold_fact = f'../data/temp/gold/fact/'

# Leitura dos Dados Modelados

In [ ]:
def extract_year_month(date): # Função para extrair o ano e o mês de uma data
    return date.strftime('%y%m')

year_month = extract_year_month(pd.to_datetime('2017-06-01'))  # Exemplo de uso da função, como se fosse o dia 01 de junho de 2017
print(f'Year-Month: {year_month}')

In [ ]:
file_name_dim_Line = 'dim_mta_PublishedLine.csv'
file_name_dim_Vehicle = 'dim_mta_VehicleRef.csv'
file_name_fact = f'fact_mta_{year_month}.csv'

In [ ]:
def read_csv_modeled_data(path_folder_gold, file_name): # Função para ler o arquivo CSV limpo
    df = pd.read_csv(os.path.join(path_folder_gold, file_name))
    
    print(f'Arquivo {file_name} lido.')
    return df

df_dim_Line = read_csv_modeled_data(path_folder_gold_dim, file_name_dim_Line)
df_dim_Vehicle = read_csv_modeled_data(path_folder_gold_dim, file_name_dim_Vehicle)
df_fact = read_csv_modeled_data(path_folder_gold_fact, file_name_fact)

display(Markdown(f'# {file_name_dim_Line}')) # Exibe o nome do arquivo com formatação Markdown de título
display(df_dim_Line.info())
display(df_dim_Line.head())

display(Markdown(f'# {file_name_dim_Vehicle}'))
display(df_dim_Vehicle.info())
display(df_dim_Vehicle.head())

display(Markdown(f'# {file_name_fact}'))
display(df_fact.info())
display(df_fact.head())

# Ajustes de tipos para inserção no Banco de Dados

In [ ]:
def format_hhmmss(td): # Função para formatar uma string de duração para o formato HH:MM:SS
    td = pd.to_timedelta(td)  # Converte a string para timedelta para usar as funções de tempo
    total_hours = td.components.hours # Obtém o total de horas
    minutes = td.components.minutes # Obtém os minutos
    seconds = td.components.seconds # Obtém os segundos
    return f"{total_hours:02d}:{minutes:02d}:{seconds:02d}" # Formata a string no formato HH:MM:SS


def change_data_types(df): # Função para alterar os tipos de dados das colunas do DataFrame
    df['RecordedAtDate'] = pd.to_datetime(df['RecordedAtDate']) # Converte a coluna RecordedAtDate para datetime
    df['RecordedAtTime'] = df['RecordedAtTime'].apply(format_hhmmss) # Converte a coluna RecordedAtTime para ser reconhecida como TIME no MySQL
    df['ScheduledArrivalTime'] = df['ScheduledArrivalTime'].apply(format_hhmmss) # Converte a coluna ScheduledArrivalTime para ser reconhecida como TIME no MySQL
    df = df.astype({'PublishedLineName': 'string', # Converte as colunas PublishedLineName e VehicleRef para string
        'VehicleRef': 'string', 
        'RecordedAtTime': 'string', 
        'ScheduledArrivalTime': 'string'}) 
    
    print("Tipos de dados das colunas alterados.")
    return df


df_dim_Line['PublishedLineName'] = df_dim_Line['PublishedLineName'].astype('string')
df_dim_Vehicle['VehicleRef'] = df_dim_Vehicle['VehicleRef'].astype('string')
df_fact = change_data_types(df_fact)

display(Markdown(f'# {file_name_dim_Line}'))
display(df_dim_Line.info())
display(df_dim_Line.head())

display(Markdown(f'# {file_name_dim_Vehicle}'))
display(df_dim_Vehicle.info())
display(df_dim_Vehicle.head())

display(Markdown(f'# {file_name_fact}'))
display(df_fact.info())
display(df_fact.head())

# Conexão com SGBD

In [ ]:
def connect_to_mysql(): # Função para conectar ao MySQL
    try:
        connection = ms.connect( # Conecta ao MySQL usando as credenciais fornecidas
            host=os.getenv('MS_HOST'),
            user=os.getenv('MS_USER'),
            password=os.getenv('MS_PWD')
        )
        print("Conexão ao MySQL estabelecida.")
        return connection
    except ms.Error as err:
        print(f"Erro ao conectar ao MySQL: {err}")
        return None
    
ms_connection = connect_to_mysql()

In [ ]:
def create_cursor(ms_connection): # Função para criar um cursor do MySQL par a executar comandos SQL
    try:
        ms_cursor = ms_connection.cursor()
        print("Cursor do MySQL criado.")
        return ms_cursor
    except:
        print("Erro ao criar o cursor do MySQL.")
        return None
    
ms_cursor = create_cursor(ms_connection)

In [ ]:
def drop_db(ms_cursor, db_name): # Função para excluir um banco de dados MySQL
    try:
        ms_cursor.execute(f"DROP DATABASE IF EXISTS {db_name}")
        print(f"Banco de dados {db_name} excluído, se existia.")
    except ms.Error as err:
        print(f"Erro ao excluir o banco de dados {db_name}: {err}")
    
db_name = 'db_bus_gps'    

drop_db(ms_cursor, db_name)

# Criação do Banco de Dados e das Tabelas

In [ ]:
def create_db(db_name, ms_cursor): # Função para criar e selecionar um banco de dados MySQL
    sql = f"CREATE DATABASE IF NOT EXISTS {db_name}"
    try:
        ms_cursor.execute(sql)
        ms_cursor.execute(f"USE {db_name}")
        print(f"Banco de dados '{db_name}' criado ou já existe, e selecionado.")
    except ms.Error as err:
        print(f"Erro ao criar o banco de dados '{db_name}': {err}.")
        
create_db(db_name, ms_cursor)

In [ ]:
def create_tb(tb_name, sql, ms_cursor): # Função para criar uma tabela no MySQL
    try:
        ms_cursor.execute(sql)
        print(f"Tabela '{tb_name}' criada ou já existe.")
    except ms.Error as err:
        print(f"Erro ao criar a tabela '{tb_name}': {err}.") 

In [ ]:
tb_name_line = 'tb_line'
        
sql_df_line = f'''
    CREATE TABLE IF NOT EXISTS {tb_name_line} (
        Published_Line_Name VARCHAR(20) NOT NULL PRIMARY KEY
    )
'''

create_tb(tb_name_line, sql_df_line, ms_cursor) # Cria a tabela tb_line

ms_cursor.execute("SHOW TABLES")
print(ms_cursor.fetchall())

In [ ]:
tb_name_vihicle = 'tb_vehicle'
        
sql_df_vehicle = f'''
    CREATE TABLE IF NOT EXISTS {tb_name_vihicle} (
        Vehicle_Ref VARCHAR(20) NOT NULL PRIMARY KEY
    )
'''

create_tb(tb_name_vihicle, sql_df_vehicle, ms_cursor) # Cria a tabela tb_vehicle

ms_cursor.execute("SHOW TABLES")
print(ms_cursor.fetchall())

In [ ]:
tb_name_bus_gps = 'tb_bus_gps'

sql_df_bus_gps = f'''CREATE TABLE IF NOT EXISTS {tb_name_bus_gps} (
        Recorded_At_Time        TIME NOT NULL,
        Direction_Ref           INT NOT NULL,
        Published_Line_Name     VARCHAR(20) NOT NULL,
        Vehicle_Ref             VARCHAR(20) NOT NULL,
        Scheduled_Arrival_Time  TIME NOT NULL,
        Recorded_A_tDate        DATE NOT NULL,
        Diff_Arrival_Mins       INT NOT NULL,
        Recorded_Time_Range     INT NOT NULL,
        Scheduled_Time_Range    INT NOT NULL,
        
        FOREIGN KEY (Vehicle_Ref) REFERENCES tb_vehicle(Vehicle_Ref),
        FOREIGN KEY (Published_Line_Name) REFERENCES tb_line(Published_Line_Name)
    )
    '''
    
create_tb(tb_name_bus_gps, tb_name_bus_gps, ms_cursor) # Cria a tabela tb_bus_gps

ms_cursor.execute("SHOW TABLES")

print(ms_cursor.fetchall()) 

# Renomeio das colunas do DataFrame para seguir o padrão do que foi criado no Banco de Dados

In [ ]:
def rename_columns(df): # Função para renomear as colunas do DataFrame para o padrão do MySQL
    columns_mapping = {
        'PublishedLineName': 'Published_Line_Name',
        'VehicleRef': 'Vehicle_Ref',
        'RecordedAtTime': 'Recorded_At_Time',
        'ScheduledArrivalTime': 'Scheduled_Arrival_Time',
        'RecordedAtDate': 'Recorded_A_tDate',
        'DiffArrivalMins': 'Diff_Arrival_Mins',
        'RecordedTimeRange': 'Recorded_Time_Range',
        'ScheduledTimeRange': 'Scheduled_Time_Range'
    }
    df = df.rename(columns=columns_mapping)
    return df

df_dim_Line = rename_columns(df_dim_Line)
df_dim_Vehicle = rename_columns(df_dim_Vehicle)
df_fact = rename_columns(df_fact)

display(df_dim_Line.columns)
display(df_dim_Vehicle.columns)
display(df_fact.columns)

# Inserção dos Dados Modelados nas Tabelas

In [ ]:
def insert_update_data(df, tb_name, ms_cursor, ms_connection): # Função para inserir dados em uma tabela MySQL
    batch_size = 200000  # Definindo o tamanho do lote para inserção

    for i in range(0, len(df), batch_size): # Itera sobre o DataFrame em lotes devido ao tamanho do DataFrame
        chunk = df.iloc[i:i + batch_size]
        
        sql = f"INSERT IGNORE INTO {tb_name} VALUES ({', '.join(['%s'] * len(chunk.columns))})"
        
        try:
            values = [tuple(row) for i, row in chunk.iterrows()]
            ms_cursor.executemany(sql, values)
            ms_connection.commit()
            print(f"{ms_cursor.rowcount} dados inseridos na tabela '{tb_name}'.")
        except ms.Error as err:
            print(f"Erro ao inserir dados na tabela {tb_name}: {err}")
            break


In [ ]:
insert_update_data(df_dim_Line, tb_name_line, ms_cursor, ms_connection) # Insere os dados na tabela tb_line

ms_cursor.execute(f"SELECT * FROM {tb_name_line} LIMIT 10")
print(ms_cursor.fetchall())

In [ ]:
insert_update_data(df_dim_Vehicle, tb_name_vihicle, ms_cursor, ms_connection) # Insere os dados na tabela tb_vehicle

ms_cursor.execute(f"SELECT * FROM {tb_name_vihicle} LIMIT 10")
print(ms_cursor.fetchall())

In [ ]:
insert_update_data(df_fact, tb_name_bus_gps, ms_cursor, ms_connection) # Insere os dados na tabela tb_bus_gps

ms_cursor.execute(f"SELECT * FROM {tb_name_bus_gps} LIMIT 10")
print(ms_cursor.fetchall())

# Pipeline de Execução

In [ ]:
def pipeline_loader_mysql(year_month): # Função para executar o pipeline de carregamento no MySQL
    print(f'Iniciando o pipeline de carregamento no MySQL')
    
    path_folder_gold_dim = '../data/temp/gold/dim/' # Caminho para a pasta de dados dimensionais
    path_folder_gold_fact = '../data/temp/gold/fact/' # Caminho para a pasta de dados factuais
    
    file_name_dim_Line = 'dim_mta_PublishedLine.csv' # Nome do arquivo CSV dimensional para linhas
    file_name_dim_Vehicle = 'dim_mta_VehicleRef.csv' # Nome do arquivo CSV dimensional para veículos
    file_name_fact = f'fact_mta_{year_month}.csv' # Nome do arquivo CSV factual para o ano-mês especificado
    
    df_dim_Line = read_csv_modeled_data(path_folder_gold_dim, file_name_dim_Line) # Lê o arquivo CSV dimensional para linhas
    df_dim_Vehicle = read_csv_modeled_data(path_folder_gold_dim, file_name_dim_Vehicle) # Lê o arquivo CSV dimensional para veículos
    df_fact = read_csv_modeled_data(path_folder_gold_fact, file_name_fact) # Lê o arquivo CSV factual para o ano-mês especificado
    
    df_dim_Line['PublishedLineName'] = df_dim_Line['PublishedLineName'].astype('string') # Converte a coluna PublishedLineName para string
    df_dim_Vehicle['VehicleRef'] = df_dim_Vehicle['VehicleRef'].astype('string') # Converte a coluna VehicleRef para string
    df_fact = change_data_types(df_fact) # Altera os tipos de dados das colunas do DataFrame factual
    
    df_dim_Line = rename_columns(df_dim_Line) # Renomeia as colunas do DataFrame dimensional para o padrão do MySQL
    df_dim_Vehicle = rename_columns(df_dim_Vehicle) # Renomeia as colunas do DataFrame dimensional para o padrão do MySQL
    df_fact = rename_columns(df_fact) # Renomeia as colunas do DataFrame factual para o padrão do MySQL
    
    ms_connection = connect_to_mysql() # Conecta ao MySQL
    ms_cursor = create_cursor(ms_connection) # Cria um cursor do MySQL para executar comandos SQL
    
    db_name = 'db_bus_gps'
    tb_name_line = 'tb_line'
    tb_name_vihicle = 'tb_vehicle'
    tb_name_bus_gps = 'tb_bus_gps'
    create_db(db_name, ms_cursor) # Cria o banco de dados db_bus_gps
    create_tb(tb_name_line, sql_df_line, ms_cursor) # Cria a tabela tb_line
    create_tb(tb_name_vihicle, sql_df_vehicle, ms_cursor) # Cria a tabela tb_vehicle
    create_tb(tb_name_bus_gps, sql_df_bus_gps, ms_cursor) # Cria a tabela tb_bus_gps
    
    insert_update_data(df_dim_Line, tb_name_line, ms_cursor, ms_connection) # Insere os dados na tabela tb_line
    insert_update_data(df_dim_Vehicle, tb_name_vihicle, ms_cursor, ms_connection) # Insere os dados na tabela tb_vehicle
    insert_update_data(df_fact, tb_name_bus_gps, ms_cursor, ms_connection) # Insere os dados na tabela tb_bus_gps
    
    print(f'Pipeline de carregamento no MySQL concluído para o ano-mês: {year_month}')
    
pipeline_loader_mysql(year_month)

In [ ]:
year_month_list = [] # Lista para armazenar os anos e meses

year_month_list.append(extract_year_month(pd.to_datetime('2017-06-01'))) # Adiciona o ano e mês de junho de 2017
year_month_list.append(extract_year_month(pd.to_datetime('2017-08-01'))) # Adiciona o ano e mês de agosto de 2017
year_month_list.append(extract_year_month(pd.to_datetime('2017-10-01'))) # Adiciona o ano e mês de outubro de 2017
year_month_list.append(extract_year_month(pd.to_datetime('2017-12-01'))) # Adiciona o ano e mês de dezembro de 2017

for year_month in year_month_list: # Itera sobre os anos e meses na lista, simulando o Airflow
    pipeline_loader_mysql(year_month)  # Executa o pipeline para o ano e mês especificados